In [1]:
# Imports
import requests
import pandas as pd
import pathlib
from urllib.request import urlretrieve

In [2]:
# Setup paths - using relative path from notebooks/ to Data/
DATA_DIR = pathlib.Path('..') / 'Data'
DATA_DIR.mkdir(exist_ok=True, parents=True)

# Get metadata

Make API Call to xeno-canto website as follows

In [3]:
def get_xc_recordings(query, page):
    """Returns data if api call works, None if it fails"""

    base_url = "https://xeno-canto.org/api/2/recordings"
    params = {"query": query, "page": page}

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception if the request fails (status code >= 400)
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        return None

#Example usage
query = "gen:Phaethornis"
get_xc_recordings(query, page=3)
get_xc_recordings(query, page=4) #Only three pages of data for this genus, so this will return None

In [4]:
retrieved_data = []
page = 1

while page_of_data := get_xc_recordings(query, page):
  retrieved_data.append(page_of_data)
  page += 1

print(f"Total number of pages: {page-1}")

Total number of pages: 3


We end up with a list of 3 dictionaries, corresponding to the 3 webpages, each of which has a key "recordings." This key holds an array with the recordings from that webpage. Let's concatenate these arrays to get one single array of recordings.

In [5]:
records = []
for page_data in retrieved_data:
    records.extend(page_data["recordings"])

Each elements of this array is a dictionary containing metadata about a single recordings. We can convert our array of dictionaries to a dataframe, where each row of the dataframe corresponds to a recording.

In [6]:
records_df = pd.DataFrame.from_dict(records)

# Check that ids are unique identifier
assert len(records_df.id.unique()) == records_df.shape[0]
# Get counts of the different species
records_df.sp.value_counts()

sp
ruber            167
eurynome         124
guy              120
longirostris     113
striigularis     103
yaruqui           87
pretrei           85
malaris           67
superciliosus     58
syrmatophorus     56
griseogularis     52
bourcieri         47
hispidus          29
nattereri         25
rupurumii         23
squalidus         21
philippii         19
atrimentalis      18
longuemareus      17
anthophilus       16
augusti           14
subochraceus       9
idaliae            7
aethopygus         6
koepckeae          5
mexicanus          5
stuarti            3
Name: count, dtype: int64

Let's download this dataframe, which contains metadata on our recordings, as a csv.

In [7]:
path = DATA_DIR / 'phaethornis_metadata.csv'
records_df.to_csv(path, index=False)
print(f"Saved metadata to {path}")

Saved metadata to ../Data/phaethornis_metadata.csv


# Download sonograms

In [8]:
records_df[records_df.sp == "augusti"].sono

451    {'small': '//xeno-canto.org/sounds/uploaded/RJ...
452    {'small': '//xeno-canto.org/sounds/uploaded/MA...
453    {'small': '//xeno-canto.org/sounds/uploaded/BD...
454    {'small': '//xeno-canto.org/sounds/uploaded/BD...
455    {'small': '//xeno-canto.org/sounds/uploaded/TG...
456    {'small': '//xeno-canto.org/sounds/uploaded/FA...
457    {'small': '//xeno-canto.org/sounds/uploaded/TN...
458    {'small': '//xeno-canto.org/sounds/uploaded/TN...
459    {'small': '//xeno-canto.org/sounds/uploaded/TN...
460    {'small': '//xeno-canto.org/sounds/uploaded/TN...
461    {'small': '//xeno-canto.org/sounds/uploaded/AF...
462    {'small': '//xeno-canto.org/sounds/uploaded/MA...
463    {'small': '//xeno-canto.org/sounds/uploaded/TN...
464    {'small': '//xeno-canto.org/sounds/uploaded/CD...
Name: sono, dtype: object

Note that we are downloading the small sonograms, which are each 240x80 pixels. If we want larger images, we can change "small" in the following code block to "medium" (480x160) or "large" (1022x396). Whichever size we choose, we will get a sonogram of the first 10 seconds of the recording. If, on the other hand, we want the sonogram of the full recording, then we should use "full". However, this will be a different sized image for each recording (since it depends on the length of the recording).





In [9]:
from fastai.vision.all import *

path = DATA_DIR / 'phaethornis_images'

for spec in records_df.sp.unique():
    dest = (path/spec)
    dest.mkdir(exist_ok=True, parents=True)
    sono_urls = ["https://"+sono_dict["small"] for sono_dict in records_df[records_df.sp == spec].sono]
    print(f"Downloading images for {spec} to {dest}")
    download_images(dest, urls=sono_urls, preserve_filename=True)

In [10]:
#Check if any image failed to download
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

0

# Download mp3 files

This takes about 15 minutes

In [11]:
path = DATA_DIR / 'phaethornis_audio'

for index, row in records_df.iterrows():
    spec = row["sp"]
    id = row["id"]
    mp3_url = row["file"]
    dest = (path/spec)
    dest.mkdir(exist_ok=True, parents=True)

    data_file_path = (dest/f"{id}.mp3")

    urlretrieve(mp3_url, data_file_path)
    if index % 50 == 0:
        print(f"Downloaded {index} mp3 files")

Downloaded 0 mp3 files
Downloaded 50 mp3 files


KeyboardInterrupt: 

# Messing around with an image classifier. Ignore this...

First we define our data format. Specifically, we specify that
- our model is an image classifier (Image ↦ Category)
- how to get the items (the function `get_image_files` returns a list of image files in a given repository)
- how to get the labels (the parent directory of the image file)
- our train-validation split (80-20)
- any transformation we want to do to our items (currently commented out because our untransformed images are all of the same 240x80 size anyways)

In [13]:
# dls = DataBlock(
#     blocks=(ImageBlock, CategoryBlock),
#     get_items=get_image_files,
#     splitter=RandomSplitter(valid_pct=0.2, seed=42),
#     get_y=parent_label
#     , #item_tfms=[Resize((240, 80), method='squish')]
# ).dataloaders(path, bs=32)

# dls.show_batch(max_n=12)

In [ ]:
# learn = vision_learner(dls, resnet18, metrics=error_rate)
# learn.fine_tune(3)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 167MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,4.445718,3.283537,0.831897,01:09


epoch,train_loss,valid_loss,error_rate,time
0,2.769383,2.621659,0.693965,01:36
1,2.086149,2.551085,0.646552,01:36
2,1.463035,2.511330,0.629310,01:35


In [14]:
# Example prediction - update path to an actual image file from your downloaded data
# example_image = list((DATA_DIR / 'phaethornis_images').rglob('*.png'))[0]
# im = PILImage.create(example_image)
# learn.predict(im)